In [1]:
import sys

sys.path.insert(0, '..')

In [2]:
from data.dataset import Dataset
from sklearn.metrics import mean_squared_error

In [13]:
TRN = Dataset.load_csv("ds/TRN_LARGE")
TST1 = Dataset.load_csv("ds/TST_1")
TST2 = Dataset.load_csv("ds/TST_2")

In [15]:
from data.featurization.dgl_Graph import DGL_Graph

featurizer = DGL_Graph(
        graph_type="BI_GRAPH",
        featurize_type="Canonical",
        self_loop=True
)
TRN.X = TRN.featurize(featurizer)
TST1.X = TST1.featurize(featurizer)
TST2.X = TST2.featurize(featurizer)

In [ ]:
from model.dgl.OGB import OGB
import torch

MODEL = OGB(
        task_type="regression",
        # OGB Configuration
        n_tasks=1,
        in_edge_feats=featurizer.get_edge_feat_size(),
        num_node_types=1,
        hidden_feats=300,
        n_layers=5,
        batchnorm=True,
        activation=torch.nn.functional.relu,
        dropout=0,
        gnn_type="gcn",
        virtual_node=True,
        residual=False,
        jk=True,
        readout="mean",
        # Abstract DGL Configuration
        lr=0.01,
        y_name="LogS exp (mol/L)",
        weight_decay=0,
        batch_size=4096,
        device=torch.device("cpu")
)
MODEL.fit(
        dataset=TRN,
        epochs=80,
        extra_eval_set=TST1,
        cv=5
)

print(f"TST1 : RMSE {mean_squared_error(TST1.y, MODEL.predict(TST1).cpu())}")
print(f"TST2 : RMSE {mean_squared_error(TST2.y, MODEL.predict(TST2).cpu())}")
print(f"TST1^: RMSE {mean_squared_error(TST1.y, MODEL.predict(TST1, True).cpu())}")
print(f"TST2^: RMSE {mean_squared_error(TST2.y, MODEL.predict(TST2, True).cpu())}")